In [76]:
import datetime

import pandas as pd
import spacy
import re
import string
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import nltk
from nltk.corpus import stopwords
from spacy.tokens import Token
from tqdm import tqdm


import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from ipywidgets import interact

%matplotlib inline
np.random.seed(500)

In [2]:
df = pd.read_csv("..//data//Womens Clothing E-Commerce Reviews Sentiment v2.csv")

In [3]:
## Discarding a empty reviews records

df=df[((df.processed_Review_text.isna()==False) & (df.processed_Review_text.isnull()==False) & (df.processed_Review_text!=""))]

In [4]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [5]:
def buildLDA(processed_docs, ip_num_topics=4):
    dictionary = gensim.corpora.Dictionary(processed_docs)

    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]

    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=ip_num_topics, id2word=dictionary, passes=2, workers=4)
    
    return dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf

In [34]:
def LDAtopicSummary(model, displaysubplot=[2,2]):
    for idx, topic in model.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))
    
#     cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

#     cloud = WordCloud(
#                       background_color='white',
#                       width=2500,
#                       height=1800,
#                       max_words=10,
#                       colormap='tab10',
#                       color_func=lambda *args, **kwargs: cols[i],
#                       prefer_horizontal=1.0)

#     topics = model.show_topics(formatted=False)

#     fig, axes = plt.subplots(displaysubplot[0], displaysubplot[1], figsize=(10,6), sharex=True, sharey=True)

#     for i, ax in enumerate(axes.flatten()):
#         #if(i<=displaysubplot[0]*displaysubplot[1]):
#             #break
#         fig.add_subplot(ax)
#         topic_words = dict(topics[i][1])
#         cloud.generate_from_frequencies(topic_words, max_font_size=300)
#         plt.gca().imshow(cloud)
#         plt.gca().set_title('Topic ' + str(i+1), fontdict=dict(size=16))
#         plt.gca().axis('off')


#     plt.subplots_adjust(wspace=0, hspace=0)
#     plt.axis('off')
#     plt.margins(x=0, y=0)
#     plt.tight_layout()
#     plt.show()

In [49]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        #print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords',"Text"]
    return(sent_topics_df)


## LDA - Non-Recommendation 

In [112]:
Neg_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]<0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_NR_processed_docs,8)
len(Neg_NR_processed_docs)

In [113]:
#LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.019505  0.006969  1       1        20.832310
4     -0.003342 -0.007929  2       1        19.474466
6     -0.006031  0.012152  3       1        17.580971
2      0.016665 -0.056594  4       1        10.268195
1      0.055094 -0.007568  5       1        10.259360
3     -0.041106  0.038866  6       1        8.841134 
7      0.047057  0.039933  7       1        7.469083 
0     -0.048832 -0.025829  8       1        5.274482 , topic_info=    Category        Freq           Term       Total  loglift  logprob
158  Default  53.000000   blouse         53.000000   30.0000  30.0000
113  Default  71.000000   pant           71.000000   29.0000  29.0000
75   Default  64.000000   chest          64.000000   28.0000  28.0000
80   Default  104.000000  short          104.000000  27.0000  27.0000
45   Default  55.000000   poor           55.000000   26.0000  26.0000
110  Default  135.000000  shirt          135.000000  25.0000  25.0000
12   Default  98.000000   tight          98.000000   24.0000  24.0000
157  Default  26.000000   stripe         26.000000   23.0000  23.0000
87   Default  72.000000   skirt          72.000000   22.0000  22.0000
152  Default  33.000000   jean           33.000000   21.0000  21.0000
89   Default  68.000000   length         68.000000   20.0000  20.0000
169  Default  43.000000   disappointing  43.000000   19.0000  19.0000
105  Default  91.000000   work           91.000000   18.0000  18.0000
57   Default  89.000000   wash           89.000000   17.0000  17.0000
30   Default  125.000000  color          125.000000  16.0000  16.0000
69   Default  107.000000  sweater        107.000000  15.0000  15.0000
81   Default  172.000000  size           172.000000  14.0000  14.0000
32   Default  63.000000   flatter        63.000000   13.0000  13.0000
182  Default  64.000000   awful          64.000000   12.0000  12.0000
29   Default  44.000000   blue           44.000000   11.0000  11.0000
76   Default  106.000000  love           106.000000  10.0000  10.0000
47   Default  111.000000  quality        111.000000  9.0000   9.0000 
88   Default  32.000000   hard           32.000000   8.0000   8.0000 
117  Default  28.000000   past           28.000000   7.0000   7.0000 
173  Default  45.000000   pattern        45.000000   6.0000   6.0000 
11   Default  179.000000  small          179.000000  5.0000   5.0000 
139  Default  33.000000   lace           33.000000   4.0000   4.0000 
6    Default  176.000000  look           176.000000  3.0000   3.0000 
78   Default  36.000000   place          36.000000   2.0000   2.0000 
172  Default  34.000000   miss           34.000000   1.0000   1.0000 
..       ...        ...    ...                 ...      ...      ... 
12   Topic8   12.894434   tight          98.151436   0.9126  -3.8367 
140  Topic8   4.439452    flat           34.430580   0.8939  -4.9029 
184  Topic8   3.779590    thigh          31.323185   0.8275  -5.0638 
198  Topic8   5.017846    maybe          42.487717   0.8061  -4.7804 
178  Topic8   3.661414    pair           31.831345   0.7797  -5.0956 
142  Topic8   6.949515    boxy           60.707123   0.7749  -4.4548 
94   Topic8   5.891835    style          51.744247   0.7695  -4.6199 
203  Topic8   3.918551    jacket         35.050434   0.7512  -5.0277 
105  Topic8   10.046503   work           91.336212   0.7350  -4.0862 
165  Topic8   2.744103    curvy          25.794983   0.7016  -5.3840 
23   Topic8   6.188984    sadly          60.931057   0.6553  -4.5707 
168  Topic8   3.036432    torso          30.204676   0.6450  -5.2828 
84   Topic8   7.078570    white          70.544189   0.6431  -4.4364 
117  Topic8   2.842424    past           28.327557   0.6431  -5.3488 
76   Topic8   10.616014   love           106.218994  0.6392  -4.0311 
32   Topic8   6.239839    flatter        63.023186   0.6297  -4.5625 
110  Topic8   12.873004   shirt          135.402832  0.58

In [125]:
Pos_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_NR_processed_docs,9)

In [126]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.008273  0.010042  1       1        22.061869
2     -0.007308  0.007996  2       1        14.045850
7      0.020414  0.031251  3       1        12.123731
1     -0.003474  0.014461  4       1        12.089794
5     -0.037610  0.006591  5       1        9.997312 
4      0.015224 -0.028520  6       1        8.750666 
8     -0.010361 -0.001997  7       1        7.623727 
0     -0.009904 -0.036323  8       1        7.305781 
6      0.041292 -0.003501  9       1        6.001263 , topic_info=    Category        Freq          Term       Total  loglift  logprob
3    Default  755.000000  dress         755.000000  30.0000  30.0000
53   Default  183.000000  button        183.000000  29.0000  29.0000
36   Default  252.000000  skirt         252.000000  28.0000  28.0000
279  Default  175.000000  unflattering  175.000000  27.0000  27.0000
159  Default  139.000000  flattering    139.000000  26.0000  26.0000
22   Default  103.000000  zipper        103.000000  25.0000  25.0000
112  Default  341.000000  beautiful     341.000000  24.0000  24.0000
219  Default  233.000000  price         233.000000  23.0000  23.0000
163  Default  224.000000  chest         224.000000  22.0000  22.0000
114  Default  489.000000  fabric        489.000000  21.0000  21.0000
40   Default  447.000000  wear          447.000000  20.0000  20.0000
381  Default  221.000000  pant          221.000000  19.0000  19.0000
49   Default  400.000000  material      400.000000  18.0000  18.0000
58   Default  179.000000  need          179.000000  17.0000  17.0000
48   Default  589.000000  look          589.000000  16.0000  16.0000
0    Default  205.000000  cheap         205.000000  15.0000  15.0000
228  Default  192.000000  expect        192.000000  14.0000  14.0000
222  Default  69.000000   snag          69.000000   13.0000  13.0000
60   Default  379.000000  short         379.000000  12.0000  12.0000
35   Default  444.000000  return        444.000000  11.0000  11.0000
214  Default  131.000000  blue          131.000000  10.0000  10.0000
25   Default  434.000000  color         434.000000  9.0000   9.0000 
77   Default  105.000000  itchy         105.000000  8.0000   8.0000 
2    Default  296.000000  design        296.000000  7.0000   7.0000 
43   Default  346.000000  feel          346.000000  6.0000   6.0000 
39   Default  203.000000  usually       203.000000  5.0000   5.0000 
122  Default  86.000000   weird         86.000000   4.0000   4.0000 
84   Default  59.000000   wool          59.000000   3.0000   3.0000 
260  Default  220.000000  boxy          220.000000  2.0000   2.0000 
108  Default  260.000000  online        260.000000  1.0000   1.0000 
..       ...         ...     ...               ...     ...      ... 
214  Topic9   18.281981   blue          131.794464  0.8379  -5.1972 
228  Topic9   24.173683   expect        192.740051  0.7371  -4.9178 
104  Topic9   21.897346   time          175.701477  0.7308  -5.0167 
39   Topic9   22.877167   usually       203.827545  0.6261  -4.9729 
3    Topic9   54.487217   dress         755.435730  0.1839  -4.1051 
40   Topic9   37.536476   wear          447.925751  0.3339  -4.4778 
163  Topic9   22.805492   chest         224.839661  0.5248  -4.9761 
129  Topic9   16.208345   gorgeous      134.857147  0.6945  -5.3176 
134  Topic9   14.157500   actually      108.691315  0.7749  -5.4528 
2    Topic9   25.314146   design        296.620361  0.3521  -4.8717 
114  Topic9   31.928551   fabric        489.620300  0.0831  -4.6396 
48   Topic9   34.607422   look          589.598633 -0.0222  -4.5590 
15   Topic9   32.894817   size          540.653015  0.0137  -4.6098 
47   Topic9   32.027901   like          541.718384 -0.0149  -4.6365 
35   Topic9   28.982393   return        444.674805  0.0825  -4.7364 
25   Topic9   27.815853   color         434.899109  0.0637  -4.7775 
255  Topic9   19.654476   wash          206.025253 

In [127]:
def demo(title):
    return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title].tolist()][:20])

df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Pos_NR_processed_docs) )
interact(demo, title=range(9));

interactive(children=(Dropdown(description='title', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0), Output()), …

## LDA - Recommended Data

In [133]:
Neg_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]<0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_R_processed_docs,10)
len(Neg_R_processed_docs)

817

In [134]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.009186 -0.043798  1       1        18.054810
0      0.024037  0.018127  2       1        14.863380
9      0.010184  0.004511  3       1        14.774433
8      0.012994 -0.019187  4       1        11.870624
5      0.010543  0.049233  5       1        9.497508 
3     -0.024732 -0.007567  6       1        9.218628 
2      0.022429  0.011462  7       1        7.996616 
6      0.046848  0.000610  8       1        7.180941 
1     -0.039481 -0.054914  9       1        4.495087 
7     -0.072009  0.041522  10      1        2.047974 , topic_info=    Category        Freq        Term       Total  loglift  logprob
50   Default  122.000000  color       122.000000  30.0000  30.0000
44   Default  85.000000   material    85.000000   29.0000  29.0000
83   Default  44.000000   pair        44.000000   28.0000  28.0000
109  Default  83.000000   jean        83.000000   27.0000  27.0000
178  Default  46.000000   blouse      46.000000   26.0000  26.0000
133  Default  36.000000   navy        36.000000   25.0000  25.0000
29   Default  98.000000   black       98.000000   24.0000  24.0000
166  Default  39.000000   boxy        39.000000   23.0000  23.0000
26   Default  34.000000   compliment  34.000000   22.0000  22.0000
2    Default  73.000000   flattering  73.000000   21.0000  21.0000
139  Default  31.000000   figure      31.000000   20.0000  20.0000
9    Default  75.000000   style       75.000000   19.0000  19.0000
136  Default  113.000000  casual      113.000000  18.0000  18.0000
115  Default  60.000000   cute        60.000000   17.0000  17.0000
164  Default  50.000000   pocket      50.000000   16.0000  16.0000
52   Default  48.000000   jacket      48.000000   15.0000  15.0000
126  Default  24.000000   expensive   24.000000   14.0000  14.0000
19   Default  168.000000  like        168.000000  13.0000  13.0000
106  Default  82.000000   soft        82.000000   12.0000  12.0000
103  Default  39.000000   winter      39.000000   11.0000  11.0000
191  Default  57.000000   blue        57.000000   10.0000  10.0000
60   Default  65.000000   pant        65.000000   9.0000   9.0000 
128  Default  30.000000   cardigan    30.000000   8.0000   8.0000 
35   Default  53.000000   price       53.000000   7.0000   7.0000 
143  Default  26.000000   seam        26.000000   6.0000   6.0000 
33   Default  64.000000   online      64.000000   5.0000   5.0000 
200  Default  23.000000   denim       23.000000   4.0000   4.0000 
171  Default  28.000000   hate        28.000000   3.0000   3.0000 
3    Default  158.000000  little      158.000000  2.0000   2.0000 
75   Default  46.000000   neck        46.000000   1.0000   1.0000 
..       ...        ...    ...              ...      ...      ... 
57   Topic10  2.461392    piece       42.031517   1.0506  -4.5709 
2    Topic10  4.290423    flattering  73.969475   1.0411  -4.0153 
93   Topic10  2.336214    button      41.474525   1.0118  -4.6231 
87   Topic10  2.599013    recommend   46.761154   0.9984  -4.5165 
161  Topic10  1.389978    appear      25.701059   0.9711  -5.1424 
188  Topic10  1.234367    spot        23.713688   0.9328  -5.2611 
84   Topic10  1.628026    stripe      32.355438   0.8989  -4.9843 
109  Topic10  4.160231    jean        83.080811   0.8941  -4.0461 
115  Topic10  2.987947    cute        60.355198   0.8827  -4.3771 
46   Topic10  2.280009    receive     47.155769   0.8590  -4.6475 
135  Topic10  1.714172    high        35.872707   0.8473  -4.9327 
27   Topic10  1.956246    especially  41.055988   0.8444  -4.8006 
33   Topic10  3.086031    online      64.848991   0.8431  -4.3448 
167  Topic10  1.349269    overall     29.095127   0.8173  -5.1721 
131  Topic10  1.751666    strap       37.881409   0.8144  -4.9111 
50   Topic10  5.580122    color       122.290672  0.8011  -3.7524 
0    Topic10  2.545007    bust        57.315239   0.7739  -4.5375 
106  Topic10  3.212853    

In [136]:
df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Neg_R_processed_docs) )
def demo(title):
    return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title].tolist()][:20])

interact(demo, title=range(10));

interactive(children=(Dropdown(description='title', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Output()…

In [137]:
Pos_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_R_processed_docs,12)
len(Pos_R_processed_docs)

17609

In [139]:
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.027310  0.006982  1       1        12.960567
7     -0.016839  0.011743  2       1        10.812213
9     -0.000302  0.005964  3       1        9.691446 
2      0.019655  0.015954  4       1        8.521266 
0     -0.011060  0.001436  5       1        8.277503 
5     -0.030247 -0.016605  6       1        8.080568 
8     -0.001760  0.003169  7       1        7.980279 
4     -0.002903  0.004194  8       1        7.402010 
3     -0.011479  0.036248  9       1        7.186934 
6     -0.010729 -0.055507  10      1        6.639723 
10     0.054771 -0.016516  11      1        6.596530 
1     -0.016418  0.002938  12      1        5.850951 , topic_info=     Category         Freq         Term        Total  loglift  logprob
232   Default  764.000000   tank         764.000000   30.0000  30.0000
642   Default  298.000000   romper       298.000000   29.0000  29.0000
135   Default  812.000000   blouse       812.000000   28.0000  28.0000
409   Default  235.000000   everyday     235.000000   27.0000  27.0000
85    Default  304.000000   form         304.000000   26.0000  26.0000
236   Default  534.000000   amazing      534.000000   25.0000  25.0000
1360  Default  197.000000   kimono       197.000000   24.0000  24.0000
147   Default  473.000000   coat         473.000000   23.0000  23.0000
870   Default  333.000000   basic        333.000000   22.0000  22.0000
261   Default  212.000000   baby         212.000000   21.0000  21.0000
61    Default  1265.000000  sleeve       1265.000000  20.0000  20.0000
32    Default  964.000000   legging      964.000000   19.0000  19.0000
151   Default  1889.000000  jean         1889.000000  18.0000  18.0000
84    Default  387.000000   fitting      387.000000   17.0000  17.0000
1446  Default  172.000000   date         172.000000   16.0000  16.0000
384   Default  466.000000   cami         466.000000   15.0000  15.0000
1244  Default  77.000000    raspberry    77.000000    14.0000  14.0000
204   Default  968.000000   jacket       968.000000   13.0000  13.0000
201   Default  965.000000   comfy        965.000000   12.0000  12.0000
22    Default  1182.000000  compliment   1182.000000  11.0000  11.0000
1543  Default  118.000000   sock         118.000000   10.0000  10.0000
214   Default  134.000000   cuter        134.000000   9.0000   9.0000 
133   Default  632.000000   tall         632.000000   8.0000   8.0000 
35    Default  1915.000000  shirt        1915.000000  7.0000   7.0000 
25    Default  2891.000000  great        2891.000000  6.0000   6.0000 
707   Default  203.000000   keeper       203.000000   5.0000   5.0000 
76    Default  678.000000   underneath   678.000000   4.0000   4.0000 
113   Default  165.000000   hour         165.000000   3.0000   3.0000 
161   Default  721.000000   nicely       721.000000   2.0000   2.0000 
698   Default  133.000000   blend        133.000000   1.0000   1.0000 
...       ...         ...     ...               ...      ...      ... 
39    Topic12  153.768555   color        2531.234131  0.0376  -4.8727 
298   Topic12  77.124954    favorite     687.608765   0.6508  -5.5627 
6     Topic12  182.587631   dress        3895.741455 -0.2218  -4.7009 
15    Topic12  134.944946   order        2156.261230  0.0673  -5.0033 
42    Topic12  140.148590   like         2361.227539  0.0143  -4.9654 
380   Topic12  113.924797   short        1669.083984  0.1541  -5.1726 
191   Topic12  88.036270    blue         950.574707   0.4592  -5.4304 
28    Topic12  139.498367   wear         2819.543701 -0.1677  -4.9701 
11    Topic12  114.288612   little       1954.307129 -0.0005  -5.1694 
45    Topic12  98.222488    pant         1432.780640  0.1584  -5.3209 
122   Topic12  88.630470    want         1119.843872  0.3021  -5.4237 
231   Topic12  111.598236   soft         2036.188721 -0.0654  -5.1932 
295   Topic12  91.449944    summer       1242.056152  0.2298  -5.3924 
133   

In [138]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Pos_R_processed_docs) )
def demo(title):
    return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title].tolist()][:20])

interact(demo, title=range(12));

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


interactive(children=(Dropdown(description='title', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=0), …